In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, KFold
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import r2_score, mean_squared_error

In [7]:
bikes_test = pd.read_excel('bike_test.xlsx')
bikes_train = pd.read_excel('bike_train.xlsx')

In [38]:
bikes_train.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [30]:
bikes_train_temp = bikes_train.copy()
bikes_train_temp = bikes_train_temp.drop(["instant", "yr", "atemp", "casual","registered","dteday"], axis=1)

In [41]:
bikes_test_temp = bikes_test.copy()
bikes_test_temp = bikes_test_temp.drop(["instant", "yr", "atemp", "dteday"], axis=1)

In [31]:
y = bikes_train_temp["cnt"]
X = bikes_train_temp.drop("cnt", axis=1)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=42)

In [33]:
xg_model = XGBRegressor()
xg_scores = cross_val_score(xg_model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=10)

In [34]:
rmse = np.sqrt(-xg_scores)
rmse

array([6.96128766, 6.96108762, 7.14637588, 7.11864735, 6.8257026 ,
       6.93436269, 7.26072291, 7.42015815, 7.17109995, 7.03594511])

In [35]:
np.mean(rmse).round()

7.0

In [36]:
xg_model.fit(X_train, y_train)
xg_model_prediction = xg_model.predict(X_test)

In [37]:
print('R2 Score: ', r2_score(y_test, xg_model_prediction)*100)
print('RMSE: ', np.sqrt(mean_squared_error(y_test, xg_model_prediction)))

R2 Score:  89.21969238508282
RMSE:  48.782666634006596


In [42]:
xg_model_prediction_proper = xg_model.predict(bikes_test_temp)

In [43]:
prototype = pd.DataFrame()
prototype['pred'] = xg_model_prediction_proper
prototype.to_csv('KevinG91.csv', index= False)